# Exemplo: Análise de dados de queimadas no Brasil

Esse notebook analisa e mostra no mapa dados sobre queimadas no Brasil.

Vamos usar os dados do INPE, recuperados em https://terrabrasilis.dpi.inpe.br/queimadas/bdqueimadas/

Os dados incluidos neste tutorial são referentes ao período de 01/07 até 01/09 de 2024.

A NASA também fornece dados de queimadas: https://firms.modaps.eosdis.nasa.gov/

In [1]:
!ls

00-Tutorial_Python_Sul_2024.md	04-Exemplo_SVD.md	     imagens
01-Tutorial_NumPy.md		05-Exemplo_Queimadas.ipynb   Untitled.ipynb
02-Tutorial_Matplotlib.md	05-Exemplo_Queimadas.md      untitled.md
03-Tutorial_SciPy.ipynb		0x-Exemplo_Masked_Arrays.md
03-Tutorial_SciPy.md		dados


In [2]:
zipfile_inpe = "dados/Focos_BDQueimadas.zip"

In [3]:
from zipfile import ZipFile

In [4]:
with ZipFile(zipfile_inpe, 'r') as zip: 
    zip.printdir() 
    print(f'Extracting file {zipfile_inpe} now...') 
    zip.extractall(path="dados") 
    print('Done!')

File Name                                             Modified             Size
focos_qmd_inpe_2024-07-01_2024-09-01_58.414406.csv 2024-09-08 16:08:08    294212594
Extracting file dados/Focos_BDQueimadas.zip now...
Done!


In [5]:
!ls dados

Focos_BDQueimadas.zip
focos_qmd_inpe_2024-07-01_2024-09-01_58.414406.csv
who_covid_19_sit_rep_time_series.csv


In [6]:
import os
csv_inpe = os.path.join("dados", "focos_qmd_inpe_2024-07-01_2024-09-01_58.414406.csv")

In [7]:
with open(csv_inpe, 'r') as f:
    data = f.readlines()

In [8]:
print(data[0:10])

['DataHora,Satelite,Pais,Estado,Municipio,Bioma,DiaSemChuva,Precipitacao,RiscoFogo,Latitude,Longitude,FRP\n', '2024/07/27 18:03:33,MSG-03,Brasil,MATO GROSSO DO SUL,AQUIDAUANA,Pantanal,,,,-19.66,-56.29,\n', '2024/07/27 18:03:33,MSG-03,Brasil,MATO GROSSO DO SUL,AQUIDAUANA,Pantanal,,,,-19.69,-56.39,\n', '2024/07/27 18:03:33,MSG-03,Brasil,MATO GROSSO DO SUL,AQUIDAUANA,Pantanal,,,,-19.69,-56.32,\n', '2024/07/27 18:03:33,MSG-03,Brasil,MATO GROSSO DO SUL,AQUIDAUANA,Pantanal,,,,-19.73,-56.41,\n', '2024/07/27 18:03:33,MSG-03,Brasil,MATO GROSSO DO SUL,AQUIDAUANA,Pantanal,,,,-19.72,-56.34,\n', '2024/07/27 18:04:59,MSG-03,Brasil,PARÁ,TRAIRÃO,Amazônia,,,,-5.03,-56.12,\n', '2024/07/27 18:04:52,MSG-03,Brasil,AMAZONAS,NOVO ARIPUANÃ,Amazônia,,,,-6.13,-59.92,\n', '2024/07/27 18:04:52,MSG-03,Brasil,AMAZONAS,NOVO ARIPUANÃ,Amazônia,,,,-6.13,-59.84,\n', '2024/07/27 18:04:52,MSG-03,Brasil,AMAZONAS,NOVO ARIPUANÃ,Amazônia,,,,-6.16,-59.93,\n']


## Pandas

Como nossos dados estão em formato de tabela, vamos usar a biblioteca [pandas](https://pydata.pandas.dev) para analisar os dados. O pandas fornece um tipo de dados chamado `DataFrame`, que permite tratar com facilidade dados em tabelas.

In [14]:
 !pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [15]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [16]:
with open(csv_inpe, 'r') as f:
    df = pd.read_csv(f)

NameError: name 'pd' is not defined

In [ ]:
df

(repare no número de linhas dessa tabela!)

Como queremos analisar dados de queimadas (representadas pelas linhas em que o valor de `RiscoFogo` é não-nulo) vamos eliminar as linhas que não nos interessam da tabela.

Aqui, temos dois tipos de dados "nulos":
- Linhas em que o valor de `RiscoFogo` é NaN (correspondente, provavelmente, a dados faltantes);
- Linhas em que o valor de `RiscoFogo` é zero.

Vamos eliminar ambos.

In [ ]:
len(df[pd.isnull(df['RiscoFogo'])])

In [ ]:
pd.isnull(df['RiscoFogo'])

Para eliminarmos essas linhas, vamos usar um filtro booleano e criar uma nova tabela com apenas os valores em que estamos interessados.

In [ ]:
df = df[~pd.isnull(df['RiscoFogo'])]

In [ ]:
df

In [ ]:
df[df['RiscoFogo']==0]

In [ ]:
df = df[df['RiscoFogo']!=0]

In [ ]:
df

Agora, vamos filtrar por satélite:

In [ ]:
df['Satelite'].unique()

In [ ]:
df = df[df['Satelite']=='TERRA_M-M']

Como todos os nossos dados são referentes ao Brasil e eliminamos os dados de outros satélites, podemos eliminar as colunas `Pais` e `Satelite` da tabela.

In [ ]:
del df['Satelite']
del df['Pais']

In [ ]:
df

Note os índices da tabela; eles ainda retém as informações da tabela original. Para resetarmos os índices, caso isso não seja necessário, usamos:

In [ ]:
df.reset_index()

e, para eliminar totalmente os índices antigos, usamos

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

**Nota:** Para saber mais sobre os valores dessa tabela, inclusive sobre como é calculado e o que significa o valor de `RiscoFogo` veja as referências abaixo:

- FRP: https://revistapesquisa.fapesp.br/como-monitorar-o-fogo/
- Risco de Queima: http://queimadas.dgi.inpe.br/queimadas/portal/informacoes/perguntas-frequentes#p23
- Monografia: https://monografias.ufrn.br/jspui/bitstream/123456789/9704/1/tcc_dias_alexandre_henrique.pdf

---

## Gráfico no mapa

Para fazermos não só um gráfico dos dados, mas sobrepor esses dados em um mapa, vamos usar a biblioteca `ipyleaflet`:

In [ ]:
# !pip install ipyleaflet

Para começarmos, vamos criar um mapa e adicionar uma camada de marcadores que serão usados mais tarde para sinalizar as coordenadas com RiscoFogo positivo.

In [ ]:
%matplotlib widget
from ipyleaflet import Map, CircleMarker

center = (-11.7997134,-53.8335376)

m = Map(center=center, zoom=3)
display(m)

In [ ]:
frp_notnull = df[df['FRP'].notnull()]
frp_notnull = frp_notnull.loc[frp_notnull['DataHora'].str.contains('2024/08/30')]

In [ ]:
for index, row in frp_notnull.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    circle_marker = CircleMarker()
    circle_marker.location = (lat, lon)
    circle_marker.radius = 1
    circle_marker.color = "red"
    circle_marker.fill_color = "red"
    circle_marker.weight = 1
    m.add_layer(circle_marker)

**Nota** Com uma instância regular do JupyterLab, você pode observar o seguinte erro ao executar o comando acima:

```
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)
```

Para resolver isso, você deve iniciar a instância do JupyterLab com o comando
```
jupyter lab --ServerApp.iopub_msg_rate_limit=10000
```
ou criar um arquivo de configurações usando o comando
```
jupyter notebook --generate-config
```
e alterar o valor dessa configuração no arquivo.

- para uma mesma cidade, pegar o risco em função do tempo
- para um grupo de cidades plotar o risco em um mesmo gráfico

In [ ]:
lista_municipios = df['Municipio'].unique()
type(lista_municipios), len(lista_municipios)

In [ ]:
pv = df[df['Municipio'] == "PORTO VELHO"]
pv

## O objeto pd.Series

In [ ]:
riscofogo = pv['RiscoFogo']
diasemchuva = pv['DiaSemChuva']

In [ ]:
type(diasemchuva)

In [ ]:
diasemchuva

O objeto `Series` é uma array unidimensional com rótulos, capaz de conter qualquer tipo de dados. Cuidado com os índices!

In [ ]:
diasemchuva[0]

In [ ]:
diasemchuva[332]

In [ ]:
riscofogo.mean()

In [ ]:
diasemchuva.max()

In [ ]:
pv[pv['DiaSemChuva'] == 88.0]

---

## Risco de fogo em função do tempo para Porto Velho, RO

In [ ]:
pv = pv[['DataHora', 'RiscoFogo', 'DiaSemChuva']].reset_index(drop=True)

In [ ]:
pv

Para cada grupo de entradas com o mesmo valor de `DataHora`, calcule a média dos valores de `RiscoFogo` e `DiaSemChuva`.

In [ ]:
agrupado = pv.groupby(['DataHora'], as_index=False)[["RiscoFogo", "DiaSemChuva"]].mean()

In [ ]:
agrupado

In [ ]:
pv[pv['DataHora']=='2024/08/22 01:52:00']

Surpresa! 🤡

In [ ]:
pv = pv[pv['RiscoFogo']!=-999.0]

In [ ]:
agrupado = pv.groupby(['DataHora'])[["RiscoFogo", "DiaSemChuva"]].mean()
agrupado

Veja https://pandas.pydata.org/docs/user_guide/groupby.html e https://pandas.pydata.org/docs/user_guide/10min.html#grouping.

In [ ]:
datas = list(agrupado.index)

In [ ]:
datas = [item[0:10] for item in datas]
datas
len(datas)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 1, figsize=(8, 6))
ax[0].plot(agrupado['RiscoFogo'], 'r')
ax[1].plot(agrupado['DiaSemChuva'], 'bo')

# Preparar rótulos da primeira imagem
ax[0].set_xticks(range(len(datas))[::10]);
ax[0].set_xticklabels(datas[::10], rotation=30)

# Preparar rótulos da segunda imagem
ax[1].set_xticks(range(len(datas))[::10]);
ax[1].set_xticklabels(datas[::10], rotation=30)

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(agrupado['RiscoFogo'], 'r')
plt.draw()

ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(agrupado['DiaSemChuva'], 'bo')

ax2.set_xticks(range(len(datas))[::10]);
ax.set_xticklabels(datas[::10], rotation=30);

ax.legend(['Risco Fogo'], loc='upper left')
ax2.legend(['Dias sem chuva'], loc='upper right')

ax.set_title('Dados INPE sobre queimadas em Porto Velho, RO')
ax.set_ylabel('Risco fogo')
ax2.set_ylabel('Dias sem chuva')
fig.tight_layout()